In [ ]:

# !wget https://raw.githubusercontent.com/jina-ai/workshops/main/memes/requirements.txt -qN
# !pip install -r requirements.txt
# !pip install pandas 
# !pip install sentence-transformers

In [22]:
from jina import Document, DocumentArray
import pandas as pd 
import random

In [77]:
def prep_docs(input_file : str, num_size = -1, shuffle=True):
    docs = DocumentArray()
    error = []
    print(f"Processing {input_file}")

    df = pd.read_csv(input_file)
    df = df.dropna().drop_duplicates()
    n = df.shape[0]
    

    if shuffle:
        df = df.sample(frac = 1)
    
    num_size = n if num_size == -1 else num_size

    for idx in range(n):
        plyr = df.iloc[idx, 3]
        psong = df.iloc[idx, 2]
        psinger = df.iloc[idx, 1]
        doctext = plyr
        if len(doctext) > 0:
            doc = Document(text=doctext)
            doc.tags['song_name'] = psong
            doc.tags['singer'] = psinger
            docs.extend([doc])
            doc = Document(text = plyr)

    return docs[:num_size]

In [78]:
docs = prep_docs(input_file= "lyrics-JINA - Sheet1.csv", num_size = 30, shuffle=True)
docs

Processing lyrics-JINA - Sheet1.csv


<jina.types.arrays.document.DocumentArray length=30 at 140420759320080>

In [79]:
#doc.tags['song_name'] = song_name


In [80]:
len(docs)

30

In [83]:
docs[0].text, docs[0].tags['song_name']

("Mmm\nHey, yeah\n(That's just for fun)\n(What?)\nI'm not one to stick around\nOne strike and you're out, baby\nDon't care if I sound crazy\nBut you never let me down, no, no\nThat's why when the sun's up, I'm stayin'\nStill layin' in your bed, singin'\nOoh, ooh, ooh, ooh\nGot all this time on my hands\nMight as well cancel our plans, yeah\nI could stay here for a lifetime\nSo lock the door\nAnd throw out the key\nCan't fight this no more\nIt's just you and me\nAnd there's nothing I, nothing I, I can do\nI'm stuck with you, stuck with you, stuck with you\nSo go ahead and drive me insane\nBaby, run your mouth\nI still wouldn't change being stuck with you\nStuck with you, stuck with you\nI'm stuck with you, stuck with you, stuck with you, baby\nThere's nowhere we need to be\nNo, no, no\nI'ma get to know you better\nKinda hope we're here forever\nThere's nobody on these streets\nIf you told me that the world's ending\nAin't no other way that I could spend it\nOh, oh, oh, oh\nGot all this 

In [84]:
for i in docs:
    if len(i.text) == 0:
        print(i.text)

In [85]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1" # Any model from Huggingface

In [86]:
from jina import Flow

In [88]:
low = (
    Flow()
    .add(
        name="lyrics_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
    )
    .add(
        name="lyrics_text_indexer",
        uses='jinahub://SimpleIndexer',
    )
)

In [89]:
flow

In [90]:
!rm -rf workspace # Remove workspace in case we've indexed stuff before

In [ ]:
with flow:
    flow.index(
        inputs=docs,
  )

Output()

Output()

2021-11-05 22:27:57.153405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-05 22:27:57.153505: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
